## OctTree!

Testing the time to look-up nearby records with the `OctTree` implementation.

The `OctTree` is used to find records within a spatio-temporal range of a given point, or within a box defined by lon, lat, & time bounds.

In [1]:
import os

os.environ["POLARS_MAX_THREADS"] = "4"

from datetime import datetime, timedelta
from string import ascii_letters, digits
import random
import inspect

import polars as pl
import numpy as np

from GeoSpatialTools.octtree import OctTree
from GeoSpatialTools.record import SpaceTimeRecord as Record
from GeoSpatialTools.shape import SpaceTimeRectangle as Rectangle

## Set-up functions

For comparisons using brute-force approach

In [2]:
def generate_uid(n: int) -> str:
    chars = ascii_letters + digits
    return "".join(random.choice(chars) for _ in range(n))

In [3]:
N = 16_000
lons = pl.int_range(-180, 180, eager=True)
lats = pl.int_range(-90, 90, eager=True)
dates = pl.datetime_range(
    datetime(1900, 1, 1, 0),
    datetime(1900, 1, 31, 23),
    interval="1h",
    eager=True,
)

lons_use = lons.sample(N, with_replacement=True).alias("lon")
lats_use = lats.sample(N, with_replacement=True).alias("lat")
dates_use = dates.sample(N, with_replacement=True).alias("datetime")
uids = pl.Series("uid", [generate_uid(8) for _ in range(N)])

df = pl.DataFrame([lons_use, lats_use, dates_use, uids]).unique()

## Add extra rows

For testing larger datasets. Uncomment to use.

In [4]:
# _df = df.clone()
# for i in range(100):
#     df2 = pl.DataFrame([
#         _df["lon"].shuffle(),
#         _df["lat"].shuffle(),
#         _df["datetime"].shuffle(),
#         _df["uid"].shuffle(),
#     ]).with_columns(
#         pl.concat_str(
#             [pl.col("uid"), pl.lit(f"{i:03d}")]
#         ).alias("uid")
#     )
#     df = df.vstack(df2)
# df.shape
# df

## Initialise the OctTree Object

In [5]:
otree = OctTree(
    Rectangle(
        -180, 180, -90, 90, datetime(1900, 1, 1, 0), datetime(1900, 1, 31, 23)
    ),
    capacity=10,
    max_depth=25,
)

In [6]:
%%time
for r in df.rows():
    otree.insert(Record(*r))

CPU times: user 104 ms, sys: 4.31 ms, total: 108 ms
Wall time: 107 ms


In [7]:
s = str(otree)
print("\n".join(s.split("\n")[:100]))

OctTree:
- boundary: SpaceTimeRectangle(west=-180, east=180, south=-90, north=90, start=datetime.datetime(1900, 1, 1, 0, 0), end=datetime.datetime(1900, 1, 31, 23, 0))
- capacity: 10
- depth: 0
- max_depth: 25
- contents:
- number of elements: 10
  * SpaceTimeRecord(x = 62, y = 5, datetime = 1900-01-23 07:00:00, uid = UeD2wZLJ)
  * SpaceTimeRecord(x = 10, y = 1, datetime = 1900-01-10 10:00:00, uid = iPFdptVo)
  * SpaceTimeRecord(x = 117, y = 5, datetime = 1900-01-09 07:00:00, uid = OyeoM1OX)
  * SpaceTimeRecord(x = -168, y = -83, datetime = 1900-01-01 04:00:00, uid = GwUhTTm6)
  * SpaceTimeRecord(x = 59, y = -49, datetime = 1900-01-05 02:00:00, uid = DLimz7ZJ)
  * SpaceTimeRecord(x = 149, y = 77, datetime = 1900-01-14 12:00:00, uid = 3MyKBX64)
  * SpaceTimeRecord(x = 174, y = -58, datetime = 1900-01-14 21:00:00, uid = 2BgYFE5T)
  * SpaceTimeRecord(x = -129, y = -67, datetime = 1900-01-31 02:00:00, uid = 4V41u23m)
  * SpaceTimeRecord(x = 159, y = -79, datetime = 1900-01-09 06:00:00, uid

## Time Execution

Testing the identification of nearby points against the original full search

In [8]:
dts = pl.datetime_range(
    datetime(1900, 1, 1),
    datetime(1900, 2, 1),
    interval="1h",
    eager=True,
    closed="left",
)
N = dts.len()
lons = 180 - 360 * np.random.rand(N)
lats = 90 - 180 * np.random.rand(N)
test_df = pl.DataFrame({"lon": lons, "lat": lats, "datetime": dts})
test_recs = [Record(*r) for r in test_df.rows()]
dt = timedelta(days=1)
dist = 350

In [9]:
%%timeit
otree.nearby_points(random.choice(test_recs), dist=dist, t_dist=dt)

189 μs ± 3.48 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
def check_cols(
    df: pl.DataFrame | pl.LazyFrame,
    cols: list[str],
    var_name: str = "dataframe",
) -> None:
    """
    Check that a dataframe contains a list of columns. Raises an error if not.

    Parameters
    ----------
    df : polars Frame
        Dataframe to check
    cols : list[str]
        Required columns
    var_name : str
        Name of the Frame - used for displaying in any error.
    """
    calling_func = inspect.stack()[1][3]
    if isinstance(df, pl.DataFrame):
        have_cols = df.columns
    elif isinstance(df, pl.LazyFrame):
        have_cols = df.collect_schema().names()
    else:
        raise TypeError("Input Frame is not a polars Frame")

    cols_in_frame = intersect(cols, have_cols)
    missing = [c for c in cols if c not in cols_in_frame]

    if len(missing) > 0:
        err_str = f"({calling_func}) - {var_name} missing required columns. "
        err_str += f"Require: {', '.join(cols)}. "
        err_str += f"Missing: {', '.join(missing)}."
        raise ValueError(err_str)

    return


def haversine_df(
    df: pl.DataFrame | pl.LazyFrame,
    lon: float,
    lat: float,
    R: float = 6371,
    lon_col: str = "lon",
    lat_col: str = "lat",
) -> pl.DataFrame | pl.LazyFrame:
    """
    Compute haversine distance on earth surface between lon-lat positions
    in a polars DataFrame and a lon-lat position.

    Parameters
    ----------
    df : polars.DataFrame
        The data, containing required columns:
            * lon_col
            * lat_col
            * date_var
    lon : float
        The longitude of the position.
    lat : float
        The latitude of the position.
    R : float
        Radius of earth in km
    lon_col : str
        Name of the longitude column
    lat_col : str
        Name of the latitude column

    Returns
    -------
    polars.DataFrame
        With additional column specifying distances between consecutive points
        in the same units as 'R'. With colname defined by 'out_colname'.
    """
    required_cols = [lon_col, lat_col]

    check_cols(df, required_cols, "df")
    return (
        df.with_columns(
            [
                pl.col(lat_col).radians().alias("_lat0"),
                pl.lit(lat).radians().alias("_lat1"),
                (pl.col(lon_col) - lon).radians().alias("_dlon"),
                (pl.col(lat_col) - lat).radians().alias("_dlat"),
            ]
        )
        .with_columns(
            (
                (pl.col("_dlat") / 2).sin().pow(2)
                + pl.col("_lat0").cos()
                * pl.col("_lat1").cos()
                * (pl.col("_dlon") / 2).sin().pow(2)
            ).alias("_a")
        )
        .with_columns(
            (2 * R * (pl.col("_a").sqrt().arcsin())).round(2).alias("_dist")
        )
        .drop(["_lat0", "_lat1", "_dlon", "_dlat", "_a"])
    )


def intersect(a, b) -> set:
    """Intersection of a and b, items in both a and b"""
    return set(a) & set(b)


def nearby_ships(
    lon: float,
    lat: float,
    pool: pl.DataFrame,
    max_dist: float,
    lon_col: str = "lon",
    lat_col: str = "lat",
    dt: datetime | None = None,
    date_col: str | None = None,
    dt_gap: timedelta | None = None,
    filter_datetime: bool = False,
) -> pl.DataFrame:
    """
    Find observations nearby to a position in space (and optionally time).

    Get a frame of all records that are within a maximum distance of the
    provided point.

    If filter_datetime is True, then only records from the same datetime will
    be returned. If a specific filter is desired this should be performed
    before calling this function and set filter_datetime to False.

    Parameters
    ----------
    lon : float
        The longitude of the position.
    lat : float
        The latitude of the position.
    pool : polars.DataFrame
        The pool of records to search. Can be pre-filtered and filter_datetime
        set to False.
    max_dist : float
        Will return records that have distance to the point <= this value.
    lon_col : str
        Name of the longitude column in the pool DataFrame
    lat_col : str
        Name of the latitude column in the pool DataFrame
    dt : datetime | None
        Datetime of the record. Must be set if filter_datetime is True.
    date_col : str | None
        Name of the datetime column in the pool. Must be set if filter_datetime
        is True.
    dt_gap : timedelta | None
        Allowed time-gap for records. Records that fall between
        dt - dt_gap and dt + dt_gap will be returned. If not set then only
        records at dt will be returned. Applies if filter_datetime is True.
    filter_datetime : bool
        Only return records at the same datetime record as the input value. If
        assessing multiple points with different datetimes, hence calling this
        function frequently it will be more efficient to partition the pool
        first, then set this value to False and only input the subset of data.

    Returns
    -------
    polars.DataFrame
        Containing only records from the pool within max_dist of the input
        point, optionally at the same datetime if filter_datetime is True.
    """
    required_cols = [lon_col, lat_col]
    check_cols(pool, required_cols, "pool")

    if filter_datetime:
        if not dt or not date_col:
            raise ValueError(
                "'dt' and 'date_col' must be provided if 'filter_datetime' "
                + "is True"
            )
        if date_col not in pool.columns:
            raise ValueError(f"'date_col' value {date_col} not found in pool.")
        if not dt_gap:
            pool = pool.filter(pl.col(date_col).eq(dt))
        else:
            pool = pool.filter(
                pl.col(date_col).is_between(
                    dt - dt_gap, dt + dt_gap, closed="both"
                )
            )

    return (
        pool.pipe(
            haversine_df,
            lon=lon,
            lat=lat,
            lon_col=lon_col,
            lat_col=lat_col,
        )
        .filter(pl.col("_dist").le(max_dist))
        .drop(["_dist"])
    )

## Generate Data

16,000 rows of data

In [11]:
def generate_uid(n: int) -> str:
    """Generates a pseudo uid by randomly selecting from characters"""
    chars = ascii_letters + digits
    return "".join(random.choice(chars) for _ in range(n))

In [12]:
N = 16_000
lons = pl.int_range(-180, 180, eager=True)
lats = pl.int_range(-90, 90, eager=True)
dates = pl.datetime_range(
    datetime(1900, 1, 1, 0),
    datetime(1900, 1, 31, 23),
    interval="1h",
    eager=True,
)

lons_use = lons.sample(N, with_replacement=True).alias("lon")
lats_use = lats.sample(N, with_replacement=True).alias("lat")
dates_use = dates.sample(N, with_replacement=True).alias("datetime")
uids = pl.Series("uid", [generate_uid(8) for _ in range(N)])

df = pl.DataFrame([lons_use, lats_use, dates_use, uids]).unique()

## Add extra rows

For testing larger datasets. Uncomment to use.

In [13]:
_df = df.clone()
for i in range(100):
    df2 = pl.DataFrame(
        [
            _df["lon"].shuffle(),
            _df["lat"].shuffle(),
            _df["datetime"].shuffle(),
            _df["uid"].shuffle(),
        ]
    ).with_columns(
        pl.concat_str([pl.col("uid"), pl.lit(f"{i:03d}")]).alias("uid")
    )
    df = df.vstack(df2)
df.shape
df

lon,lat,datetime,uid
i64,i64,datetime[μs],str
49,31,1900-01-14 17:00:00,"""J91jqL2U"""
144,-10,1900-01-02 08:00:00,"""a8suYytS"""
57,4,1900-01-11 23:00:00,"""OgycUSlv"""
-91,89,1900-01-16 13:00:00,"""kOwAhzlv"""
110,-12,1900-01-26 19:00:00,"""6pL5aIcy"""
…,…,…,…
35,-46,1900-01-03 11:00:00,"""CpYzLZKL099"""
-146,-11,1900-01-17 03:00:00,"""g6zW2XoY099"""
63,-71,1900-01-02 05:00:00,"""qwgwyHRr099"""


## Initialise the OctTree Object

There is an overhead in constructing the `OctTree` class. The performance benefits appear if multiple neighbourhood searches are performed.

In [14]:
bounds = Rectangle(
    -180,
    180,
    -90,
    90,
    datetime(1900, 1, 1, 0),
    datetime(1900, 1, 31, 23),
)
otree = OctTree(bounds, capacity=10, max_depth=25)

In [15]:
%%time
for r in df.rows():
    otree.insert(Record(*r))

CPU times: user 15.2 s, sys: 341 ms, total: 15.6 s
Wall time: 15.5 s


### View the `OctTree`

It is a nested object, with child `OctTree` objects in each octant of the space-time domain.

In [16]:
s = str(otree)
print("\n".join(s.split("\n")[:100]))

OctTree:
- boundary: SpaceTimeRectangle(west=-180, east=180, south=-90, north=90, start=datetime.datetime(1900, 1, 1, 0, 0), end=datetime.datetime(1900, 1, 31, 23, 0))
- capacity: 10
- depth: 0
- max_depth: 25
- contents:
- number of elements: 10
  * SpaceTimeRecord(x = 49, y = 31, datetime = 1900-01-14 17:00:00, uid = J91jqL2U)
  * SpaceTimeRecord(x = 144, y = -10, datetime = 1900-01-02 08:00:00, uid = a8suYytS)
  * SpaceTimeRecord(x = 57, y = 4, datetime = 1900-01-11 23:00:00, uid = OgycUSlv)
  * SpaceTimeRecord(x = -91, y = 89, datetime = 1900-01-16 13:00:00, uid = kOwAhzlv)
  * SpaceTimeRecord(x = 110, y = -12, datetime = 1900-01-26 19:00:00, uid = 6pL5aIcy)
  * SpaceTimeRecord(x = 132, y = 17, datetime = 1900-01-27 15:00:00, uid = cs22YTNi)
  * SpaceTimeRecord(x = -126, y = 42, datetime = 1900-01-09 20:00:00, uid = iF0xl2If)
  * SpaceTimeRecord(x = -4, y = 19, datetime = 1900-01-16 07:00:00, uid = OP9LywAR)
  * SpaceTimeRecord(x = 78, y = -48, datetime = 1900-01-15 09:00:00, uid =

## Time Execution

Testing the identification of nearby points against the original full search

In [17]:
dts = pl.datetime_range(
    datetime(1900, 1, 1),
    datetime(1900, 2, 1),
    interval="1h",
    eager=True,
    closed="left",
)
N = dts.len()
lons = 180 - 360 * np.random.rand(N)
lats = 90 - 180 * np.random.rand(N)
test_df = pl.DataFrame({"lon": lons, "lat": lats, "datetime": dts})
test_recs = [Record(*r) for r in test_df.rows()]
dt = timedelta(days=1)
dist = 350

In [18]:
%%timeit
otree.nearby_points(random.choice(test_recs), dist=dist, t_dist=dt)

1.18 ms ± 32.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%%timeit
rec = random.choice(test_recs)
nearby_ships(
    lon=rec.lon,
    lat=rec.lat,
    dt=rec.datetime,
    max_dist=dist,
    dt_gap=dt,
    date_col="datetime",
    pool=df,
    filter_datetime=True,
)

11.7 ms ± 114 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Verify

Check that records are the same

In [20]:
%%time
dist = 250
for _ in range(250):
    rec = Record(*random.choice(df.rows()))
    orig = nearby_ships(
        lon=rec.lon,
        lat=rec.lat,
        dt=rec.datetime,
        max_dist=dist,
        dt_gap=dt,
        date_col="datetime",
        pool=df,
        filter_datetime=True,
    )  # noqa
    tree = otree.nearby_points(rec, dist=dist, t_dist=dt)
    if orig.height > 0:
        if not tree:
            print(rec)
            print("NO TREE!")
            print(f"{orig = }")
        else:
            tree = pl.from_records(
                [(r.lon, r.lat, r.datetime, r.uid) for r in tree], orient="row"
            ).rename(
                {
                    "column_0": "lon",
                    "column_1": "lat",
                    "column_2": "datetime",
                    "column_3": "uid",
                }
            )  # noqa
            if tree.height != orig.height:
                print("Tree and Orig Heights Do Not Match")
                print(f"{orig = }")
                print(f"{tree = }")
            else:
                # tree = tree.with_columns(pl.col("uid").str.slice(0, 6))
                if not tree.sort("uid").equals(orig.sort("uid")):
                    print("Tree and Orig Do Not Match")
                    print(f"{orig = }")
                    print(f"{tree = }")

CPU times: user 2min 10s, sys: 9.59 s, total: 2min 20s
Wall time: 2min 17s


## Check -180/180 boundary

Expect to see `SpaceTimeRecord`s with both positive and negative `"x"` values

In [21]:
out = otree.nearby_points(
    Record(179.5, -43.1, datetime(1900, 1, 14, 13)),
    dist=200,
    t_dist=timedelta(days=3),
)
for o in out:
    print(o)

SpaceTimeRecord(x = -180, y = -44, datetime = 1900-01-11 13:00:00, uid = 9cex7Wyj040)
SpaceTimeRecord(x = -180, y = -44, datetime = 1900-01-11 13:00:00, uid = HUjx4OhG057)
SpaceTimeRecord(x = -180, y = -43, datetime = 1900-01-12 00:00:00, uid = DHIjHUru002)
SpaceTimeRecord(x = -180, y = -42, datetime = 1900-01-11 22:00:00, uid = x53Sdbaq007)
SpaceTimeRecord(x = -180, y = -44, datetime = 1900-01-11 17:00:00, uid = Bo4Bf7ez039)
SpaceTimeRecord(x = -180, y = -44, datetime = 1900-01-12 03:00:00, uid = Kgjv5F8M063)
SpaceTimeRecord(x = -180, y = -43, datetime = 1900-01-13 01:00:00, uid = lpQi0aLa011)
SpaceTimeRecord(x = -179, y = -42, datetime = 1900-01-13 01:00:00, uid = M61uNtKi072)
SpaceTimeRecord(x = -180, y = -42, datetime = 1900-01-13 01:00:00, uid = I3scgwwG094)
SpaceTimeRecord(x = -180, y = -43, datetime = 1900-01-12 19:00:00, uid = 3cYRiCls057)
SpaceTimeRecord(x = -180, y = -43, datetime = 1900-01-13 04:00:00, uid = qVWUrKKg045)
SpaceTimeRecord(x = -179, y = -43, datetime = 1900-01-